# Identificação de tópicos quentes na computação através da análise em titulos de artigos.

Modelo ainda preliminar. Tentar utilizar os modelos LDA e LSI de forma conjunta, produzindo tópicos híbridos. Tentar obter palavras chave do artigo... mais eficiente

Primeiro fazemos os imports - nltk (porter stemmer e stopwords)) e gensim (modelagem de tópicos)

In [2]:
import sys
import time
import os
import re
import operator
import matplotlib.pyplot as plt
import warnings
import gensim
import numpy as np
warnings.filterwarnings('ignore')  

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel, TfidfModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
from pprint import pprint

from gensim.utils import lemmatize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


%matplotlib inline

#### Parseando a dblp

As classes abaixo são responsáveis por ler a dblp, extraindo para cada um de seus artigos o nome, o titulo, o ano e os autores. O campo title armazena o array de tokens utilizados com o devido pré-processamento (lowercase, split, remoção de stopwords e finalmente stemming). Note que o tempo de parsing da dblp é pequeno

In [3]:
stemmer = PorterStemmer()
stops = set(stopwords.words('english'))

class Artigo(object):
    
    def __init__(self):
        self.year = None
        self.title = None
        self.journal = None
        self.author = []
        self.originalTitle = None
        
    def add_author(self,author):
        self.author.append(author.decode('utf8').replace('<author>', '').replace('</author>', '').strip())
        
    def add_year(self, year):
        self.year = year.decode('utf8').replace('<year>', '').replace('</year>', '').strip()
    
    def add_title(self, title):
        self.originalTitle = title.decode('utf8').replace('<title>', '').replace('</title>', '').strip()
        texto = gensim.utils.simple_preprocess(self.originalTitle, deacc=True, min_len=2)
        texto = [stemmer.stem(word) for word in texto if word not in stops]     
        self.title = texto
        
    
    def add_journal(self, journal):
        self.journal = journal.decode('utf8').replace('<journal>', '').replace('</journal>', '').strip()
    
    def validate(self):
        return not self.year is None and not self.title is None and not self.journal is None
    
class Artigos(object):
    
    def __init__(self, arr=[]):
        self.artigos = arr
        
        
    def add_artigo(self, artigo):
        self.artigos.append(artigo)
        
    def __iter__(self):
        for artigo in self.artigos:
            yield artigo
            
    def get_textos(self):
        for artigo in self.artigos:
            yield artigo.title
            
    def aplicar_bigramas(self):
        self.bigram = gensim.models.Phrases(artigos.get_textos())
        for artigo in self.artigos:
            artigo.title = self.bigram[artigo.title]
    
    def get_training_text(self):
        arr = []
        for texto in self.get_textos():
            arr.append(texto)
        return arr
            
    
def extract_artigos(file="/Volumes/My Passport/phd/dblp.xml"):
    artigos = Artigos()
    with open(file, "r") as ins:
        i = 0
        artigoAtual = None
        for line in ins:
            if '<article' in line:    
                if not artigoAtual is None and artigoAtual.validate():
                    artigos.add_artigo(artigoAtual)
                artigoAtual = Artigo()
                i+=1
                if i % 1000 == 0:
                    sys.stdout.write("\r" + str(i))
                    sys.stdout.flush()
                if i > 60000:
                    break
            else:
                if line.startswith('<author>'):
                    artigoAtual.add_author(line)
                if line.startswith('<year>'):
                    artigoAtual.add_year(line)
                if line.startswith('<title>'):
                    artigoAtual.add_title(line)
                if line.startswith('<journal>'):
                    artigoAtual.add_journal(line)

    ins.close()
    return artigos

ts = time.time()
artigos = extract_artigos()
te = time.time()

print ''
print 'finalizado em {tempo}'.format(tempo=(te-ts))
print len(artigos.artigos)

60000
finalizado em 22.9328019619
60000


Nesta etapa identificamos os brigramas existentes, efetuando o merge diretamente no campo title.

In [4]:
artigos.aplicar_bigramas()

Abaixo vemos os dez primeiros exemplos de titulos originais e o resultado final da nossa tokenização

In [5]:
i =0
for artigo in artigos:
    print artigo.originalTitle
    print artigo.title
    print 
    i+=1
    if i > 10:
        break

Parallel Integer Sorting and Simulation Amongst CRCW Models.
[u'parallel', u'integ', u'sort', u'simul', u'amongst', u'crcw', u'model']

Pattern Matching in Trees and Nets.
[u'pattern_match', u'tree', u'net']

NP-complete Problems Simplified on Tree Schemas.
[u'np_complet', u'problem', u'simplifi', u'tree', u'schema']

On the Power of Chain Rules in Context Free Grammars.
[u'power', u'chain', u'rule', u'context_free', u'grammar']

Schnelle Multiplikation von Polynomen &uuml;ber K&ouml;rpern der Charakteristik 2.
[u'schnell', u'multiplik', u'von', u'polynomen', u'uuml', u'ber', u'ouml', u'rpern', u'der', u'charakteristik']

A characterization of rational D0L power series.
[u'ration', u'power', u'seri']

The Derivation of Systolic Implementations of Programs.
[u'deriv', u'systol', u'implement', u'program']

Fifo Nets Without Order Deadlock.
[u'fifo', u'net', u'without', u'order', u'deadlock']

On the Complementation Rule for Multivalued Dependencies in Database Relations.
[u'complement', 

Abaixo os bigramas encontrados ao longo do corpus

In [6]:
i =0
for item in artigos.bigram.vocab:
    if '_' in item:
        print item
    i+=1
    if i > 10:
        break
        
# ver depois
    

build_parallel
signal_concept
network_variat
touch_dynam
contrast_polar
accuraci_trade
reduct_variabl
use_zone
ray_fluoresc
parametr_maximum
scheme_without


#### Modelagem de tópicos


Agora capturamos os titulos dos artigos em um array de treinamento da modelagem de tópico.

In [7]:
training_texts = artigos.get_training_text()
print training_texts[0:2]

[[u'parallel', u'integ', u'sort', u'simul', u'amongst', u'crcw', u'model'], [u'pattern_match', u'tree', u'net']]


Criando os dicionarios e corpus devidamente convertido para a sua representação numérica.

In [8]:
dictionary = Dictionary(training_texts)
corpus = [dictionary.doc2bow(text) for text in training_texts]

Neste trecho, treinamos o modelo LDA (Latent Dirichlet Allocation). Note que para um subset com 60000 titulos de artigos, o tempo de treinamento foi de apenas 3 minutos em uma máquina comum (i7 com 8gb de ram). 

A quantidade de tópicos precisa ser informada de forma a maximizar a coerência do corpus. Desta forma, múltiplos experimentos devem ser executados de forma a encontrar o valor ideal. Após alguns testes preliminares, decidimos utilizar 150 tópicos.

O resultado do modelo é registrado em arquivo próprio.

In [9]:
ts = time.time()
ldaModel = LdaModel(corpus=corpus, id2word=dictionary, num_topics=150)
ldaModel.save('/Volumes/My Passport/phd/ldamodel')
te = time.time()
print 'modelo treinado em {tempo}'.format(tempo=(te-ts))

modelo treinado em 209.071384192


In [9]:
ldaModel = LdaModel.load('/Volumes/My Passport/phd/ldamodel')

Abaixo medimos a coerencia do modelo com os dados do próprio treinamento. Em seguida, listamos as principais palavras existentes em cada tópico.

In [10]:
cm = CoherenceModel(model=ldaModel, corpus=corpus, dictionary=dictionary, texts=training_texts)
print cm.get_coherence()

0.34835663341270884


In [11]:
ldaModel.print_topics()

[(125,
  u'0.132*"analysi" + 0.103*"uncertainti" + 0.102*"decentr" + 0.092*"cloud" + 0.058*"driven" + 0.048*"execut" + 0.046*"critic" + 0.039*"record" + 0.034*"model" + 0.031*"privat"'),
 (63,
  u'0.108*"threshold" + 0.098*"compon" + 0.088*"noisi" + 0.072*"norm" + 0.054*"famili" + 0.038*"mous" + 0.031*"affect" + 0.030*"vs" + 0.026*"base" + 0.022*"use"'),
 (18,
  u'0.230*"multivari" + 0.213*"knowledg" + 0.137*"system" + 0.032*"model" + 0.027*"taxonomi" + 0.020*"do" + 0.012*"musculoskelet" + 0.011*"analysi" + 0.010*"dead_time" + 0.009*"data"'),
 (32,
  u'0.191*"random" + 0.170*"fingerprint" + 0.122*"semant" + 0.035*"align" + 0.031*"analysi" + 0.027*"java" + 0.021*"base" + 0.017*"system" + 0.016*"minutia" + 0.016*"model"'),
 (52,
  u'0.122*"understand" + 0.082*"gpu" + 0.044*"use" + 0.041*"correspond" + 0.035*"compress_sens" + 0.035*"gate" + 0.033*"post" + 0.032*"scenario" + 0.025*"cerebr" + 0.024*"asymptot_stabil"'),
 (148,
  u'0.104*"extend" + 0.054*"hash" + 0.040*"petri_net" + 0.040*"fo

O trecho abaixo verifica para cada documento os tópicos aos quais ele está associado.

In [12]:
outro_bow = [dictionary.doc2bow(text) for text in training_texts[0:10]]
probGrupo = ldaModel[outro_bow]
for prob in probGrupo:
    grupos = []
    for item in prob:
        grupos.append(item[0])
    print grupos

[26, 31, 53, 87, 119]
[49, 131, 148]
[31, 32, 104, 131, 134]
[49, 62, 65, 69, 77]
[13, 66, 115, 135, 139]
[25, 35, 62]
[26, 53, 54, 101]
[4, 13, 49, 102, 110]
[11, 77, 82, 97, 149]
[72, 103, 108, 131]


In [13]:
print ldaModel.show_topic(131, 30)

[(u'featur', 0.2602153178916348), (u'use', 0.14999187482628407), (u'base', 0.07220998516021344), (u'tree', 0.0656962648849924), (u'ratio', 0.03046064437385665), (u'thyroid', 0.023848613312963094), (u'boost', 0.022553830386287426), (u'nonparametr', 0.02121605069435915), (u'biomed', 0.019166254841288775), (u'featur_select', 0.016491278712993163), (u'imag_hash', 0.014250874692214869), (u'mixtur', 0.013545874530657797), (u'ga', 0.012677081897783896), (u'specifi', 0.012357526480068177), (u'basic', 0.011925863316429516), (u'prostat', 0.01177305574857011), (u'ultrasound_imag', 0.011584949063772594), (u'signatur_verif', 0.010972513446288685), (u'ode', 0.010661956932903506), (u'system', 0.00981525049055447), (u'data', 0.008748743253872824), (u'gabor', 0.008513274329816117), (u'arrang', 0.007222273013596076), (u'spoof_detect', 0.007094268624770328), (u'lbp', 0.004917537676838191), (u'discharg', 0.0045200634659358085), (u'ambigu', 0.003923321583542993), (u'esprit', 0.00389586549192057), (u'curvel

In [ ]:
lsimodel = LsiModel(corpus=corpus, num_topics=100, id2word=dictionary)
print lsimodel.show_topics(num_topics=5)